In [112]:
import tushare as ts
import pandas as pd
from datetime import datetime 
pro = ts.pro_api()

In [100]:
def zscore(ts_code):
    df_balance = pro.balancesheet(ts_code=ts_code,end_type=4)[:1]
    df_balance.fillna(0)
    df_income = pro.income(ts_code=ts_code,end_type=4)[:1]
    df_income=df_income.fillna(0)
    df_basic = pro.daily_basic(ts_code=ts_code)[:1]
    
    total_assets=df_balance['total_assets'].astype('float').values[0]

    #A= 流动资本/总资产 = (流动资产-流动负债)/总资产 
    total_cur_assets=df_balance['total_cur_assets'].astype('float').values[0]
    total_cur_liab=df_balance['total_cur_liab'].astype('float').values[0]
    A=(total_cur_assets-total_cur_liab)/total_assets
    print("A="+str(A))
    
    
    #B = 留存收益/总资产 =(股东权益合计-股本)/总资产 
    total_hldr_eqy_inc_min_int =df_balance['total_hldr_eqy_inc_min_int'].astype('float').values[0]
    total_share =df_balance['total_share'].astype('float').values[0]
    B=(total_hldr_eqy_inc_min_int-total_share)/total_assets
    print("B="+str(B))
    
    #C = 息税前收益/总资产 =(利润总额+财务费用)/总资产 
    total_profit = df_income['total_profit'].astype('float').values[0]
    rd_exp = df_income['rd_exp'].astype('float').values[0]
    C=(total_profit+rd_exp)/total_assets
    print("C="+str(C))
    
    #D = 优先股和普通股市值/总负债=(股价*股票总数)/总负债 
    total_mv=df_basic['total_mv'].astype('float').values[0]
    total_liab=df_balance['total_liab'].astype('float').values[0]
    D=total_mv/total_liab
    print("D="+str(D))
    
    #E= 销售额/总资产
    revenue=df_income['revenue'].astype('float').values[0]
    E=revenue/total_assets
    print("E="+str(E))
    
    Z=1.2*A + 1.4*B + 3.3*C + 0.6*D + 0.99*E
    print("Z="+str(Z))
    
    if(Z < 1.8):
        print(ts_code+'当前财务状况为破产区')
    elif (Z > 2.99):
        print(ts_code+'当前财务状况为安全区')
    else:
        print(ts_code+'当前财务状况为灰色区')

zscore('002624.SZ') #完美世界
zscore('300029.SZ') #ST天龙

A=0.2373720791211451
B=0.5960189888993613
C=0.20931873421993563
D=0.000552221072677601
E=0.659367585411381
Z=2.463130144531141
002624.SZ当前财务状况为灰色区
A=-0.26442297420871724
B=-0.8405015431764118
C=-0.134858255018074
D=0.0005087275380955364
E=0.5096087724880183
Z=-1.434224049771086
300029.SZ当前财务状况为破产区


In [99]:
import math
from math import e
def oscore(ts_code):
    df_balance = pro.balancesheet(ts_code=ts_code,end_type=4)[:1]
    df_income = pro.income(ts_code=ts_code,end_type=4)[:1]
    df_basic = pro.daily_basic(ts_code=ts_code)[:1]
    df_ind = pro.fina_indicator(ts_code=ts_code,end_type=4)[:1]
    df_cashflow=pro.cashflow(ts_code=ts_code,end_type=4)[:1]
    
    # TA(Total Assets) = 总资产
    TA=df_balance['total_assets'].astype('float').values[0]    
    print("TA="+str(TA))
    
    # GNP(gross national product price index level) = 国民生产总值价格指数水平
    GNP=700
    print("GNP="+str(GNP))
    
    # TL(Total Liabilities) = 总负债
    TL=df_balance['total_liab'].astype('float').values[0]
    print("TL="+str(TL))
    
    # WC(Working Capital) = 营运资金
    WC=df_ind['working_capital'].astype('float').values[0]
    print("WC="+str(WC))
    
    # CL(Current Liabilities) = 流动负债
    CL=df_balance['total_cur_liab'].astype('float').values[0]
    print("CL="+str(CL))
    
    # CA(Current Assets) = 流动资产
    CA=df_balance['total_cur_assets'].astype('float').values[0]
    print("CA="+str(CA))
    
    # X = 如果 总负责TL>总资产TA，则为1，否则为 0
    X=1 if TL>TA else 0
    print("X="+str(X))
    
    # NI(Net income) = 净收入
    NI=df_income['n_income'].astype('float').values[0]
    print("NI="+str(NI))
    
    # FFO(Funds From Operations) = 运营资金
    # FFO = 净收入 +（折旧费用 + 摊销费用 + 资产出售损失）-（资产出售收益 + 利息收入）
    depr_fa_coga_dpba=df_cashflow['depr_fa_coga_dpba'].astype('float').values[0]
    amort_intang_assets=df_cashflow['amort_intang_assets'].astype('float').values[0]
    FFO=NI #上面那些费用取不出来
    print("FFO="+str(FFO))
    
    # Y = 如果过去两年净亏损，则为1，否则为0
    Y=0 #懒得算了，先写成0
    
    T=-1.32-0.407*math.log(TA/GNP)+6.03*(TL/TA)-1.43*(WC/TA)+0.0757*(CL/CA)-1.72*X-2.37*(NI/TA)-1.83*(FFO/TL)+0.285*Y-0.521*0
    prob=(e**T)/(1+e**T)
    print("T="+str(T))
    
    print(ts_code+"破产的概率为:"+str(round(prob*100,2))+"%")

oscore('002624.SZ') #完美世界
oscore('300029.SZ') #ST天龙

TA=15506930273.09
GNP=700
TL=4324538166.79
WC=3360571013.16
CL=4203704700.16
CA=7884616979.87
X=0
NI=1504287486.34
FFO=1504287486.34
T=-7.658163712052047
002624.SZ破产的概率为:0.05%
TA=220807920.85
GNP=700
TL=206397319.07
WC=-86224358.95
CL=206391413.11
CA=148004725.95
X=0
NI=-31974400.4
FFO=-31974400.4
T=0.45380180114493657
300029.SZ破产的概率为:61.15%


In [111]:
def mscore(ts_code):
    df_balance = pro.balancesheet(ts_code=ts_code,end_type=4)
    df_balance=df_balance.drop_duplicates('end_date', keep='first', inplace=False)[:2]
    df_income = pro.income(ts_code=ts_code,end_type=4)
    df_income=df_income.drop_duplicates('end_date', keep='first', inplace=False)[:2]
    df_basic = pro.daily_basic(ts_code=ts_code,end_type=4)
    df_ind = pro.fina_indicator(ts_code=ts_code,end_type=4)
    df_ind=df_ind.drop_duplicates('end_date', keep='first', inplace=False)[:2]
    df_cashflow=pro.cashflow(ts_code=ts_code,end_type=4)
    df_cashflow=df_cashflow.drop_duplicates('end_date', keep='first', inplace=False)[:2]
    
    df_balance=df_balance.fillna(0)
    df_cashflow=df_cashflow.fillna(0)
    df_income=df_income.fillna(0)
    df_ind=df_ind.fillna(0)
    
    # DSRI =（ 本期 应收账款 / 营业收入 ）/（ 上期 应收账款 / 营业收入 ）
    accounts_receiv=df_balance['accounts_receiv'].astype('float').values
    revenue=df_income['revenue'].astype('float').values
    DSRI=(accounts_receiv[0]/revenue[0])/(accounts_receiv[1]/revenue[1])
    print("DSRI="+str(DSRI))
    
    # GMI = 上期 毛利率 ％ / 本期 毛利率 ％
    grossprofit_margin=df_ind['grossprofit_margin'].astype('float').values
    GMI=grossprofit_margin[0]/grossprofit_margin[1]
    print("GMI="+str(GMI))
    
    # AQI = 本期 资产质量比率 / 上期 资产质量比率
    # 资产质量比率 = 1 -（ 流动资产合计 + 固定资产、无形资产和其他长期资产净额 ）/ 资产总计
    total_cur_assets=df_balance['total_cur_assets'].astype('float').values
    fix_assets=df_balance['fix_assets'].astype('float').values
    intan_assets=df_balance['intan_assets'].astype('float').values
    oth_nca=df_balance['oth_nca'].astype('float').values
    total_assets=df_balance['total_assets'].astype('float').values
    AQ0=1-(total_cur_assets[0]+fix_assets[0]+intan_assets[0]+oth_nca[0])/total_assets[0]
    AQ1=1-(total_cur_assets[1]+fix_assets[1]+intan_assets[1]+oth_nca[1])/total_assets[1]
    AQI=AQ0/AQ1
    if math.isnan(AQI):
        AQI=1
    print("AQI="+str(AQI))
    
    # SGI = 本期 营业收入 / 上期 营业收入
    SGI=revenue[0]/revenue[1]
    print("SGI="+str(SGI))
    
    # DEPI = 上期 折旧率 / 本期 折旧率
    # 折旧率 = 折旧、损耗和摊销 /（ 折旧、损耗和摊销 + 固定资产、无形资产和其他长期资产净额 ）
    depr_fa_coga_dpba=df_cashflow['depr_fa_coga_dpba'].astype('float').values
    DEP0=depr_fa_coga_dpba[0]/(depr_fa_coga_dpba[0]+total_cur_assets[0]+fix_assets[0]+intan_assets[0]+oth_nca[0])
    DEP1=depr_fa_coga_dpba[1]/(depr_fa_coga_dpba[1]+total_cur_assets[1]+fix_assets[1]+intan_assets[1]+oth_nca[1])
    DEPI=DEP0/DEP1
    print("DEPI="+str(DEPI))
    
    # SGAI =（ 本期 销售及行政开支 / 营业收入 ）/（ 上期 销售及行政开支 / 营业收入 ）
    sell_exp=df_income['sell_exp'].astype('float').values
    admin_exp=df_income['admin_exp'].astype('float').values
    fin_exp=df_income['fin_exp'].astype('float').values
    SGA0=(sell_exp[0]+admin_exp[0]+fin_exp[0])/revenue[0]
    SGA1=(sell_exp[1]+admin_exp[1]+fin_exp[1])/revenue[1]
    SGAI=SGA0/SGA1
    print("SGAI="+str(SGAI))
    
    #LVGI = 本期 资产负债率 / 上期 资产负债率
    #资产负债率 =（ 长期借款和资本化租赁债务 + 流动负债合计 ）/ 资产总计
    debt_to_assets=df_ind['debt_to_assets'].astype('float').values
    LVGI=debt_to_assets[0]/debt_to_assets[1]
    print("LVGI="+str(LVGI))
    
    #TATA =（ 归属于母公司股东的净利润 - 非营业利润 - 经营活动产生的现金流量净额 ）/ 资产总计
    profit_dedt=df_ind['profit_dedt'].astype('float').values[0]
    n_cashflow_act=df_cashflow['n_cashflow_act'].astype('float').values[0]
    TATA=(profit_dedt-n_cashflow_act)/total_assets[0]
    print("TATA="+str(TATA))
    
    MScore=-4.840+0.920*DSRI+0.528*GMI+0.404*AQI+0.892*SGI+0.115*DEPI-0.172*SGAI -0.327*LVGI+4.697*TATA
    print("MScore="+str(MScore))
    
    if(MScore>-1.78):
         print(ts_code+'财务造假嫌疑高')
    else:
         print(ts_code+'财务造假嫌疑低')
mscore('002624.SZ') #完美世界
mscore('300029.SZ') #ST天龙

DSRI=0.44999269262861763
GMI=1.0844061668238596
AQI=1.2324143324865937
SGI=1.271892039683023
DEPI=1.2404320042147092
SGAI=1.0431665981294453
LVGI=0.9351976535085863
TATA=-0.201923738785604
MScore=-3.512037585143696
002624.SZ财务造假嫌疑低
DSRI=121.75280767309881
GMI=1.0694407447390188
AQI=1
SGI=6.547216265692907
DEPI=0.3767387379625849
SGAI=0.10696148434728352
LVGI=1.0824958669207672
TATA=-0.00241144216181318
MScore=113.64098956871203
300029.SZ财务造假嫌疑高


/tmp/ipykernel_25942/3186124987.py:37: RuntimeWarning: invalid value encountered in double_scalars
  AQI=AQ0/AQ1
